In [ ]:
# This requirement has been installed in assignment 0
from furhat_remote_api import FurhatRemoteAPI
furhat = FurhatRemoteAPI("localhost")

In [ ]:
furhat.set_face(character='Isabel', mask="Adult")
furhat.set_voice(name='Joanna')

In [ ]:
import cv2
def image_capture(save_path):
    cap = cv2.VideoCapture(0)
    # cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # Use DirectShow backend on Windows
    if not cap.isOpened():
        print("Cannot open camera")
        return None
    
    ok, pic = cap.read()
    cap.release()
            
    if not ok:
        print("Cannot read frame")
        return None
    cv2.imwrite(save_path, pic)
    print(f"Image saved to {save_path}")
    return pic

image_capture("webcam_image.jpg")

In [ ]:
from google import genai
import os
from google.genai import types
def open_gemini(system_instruction: str):
    API_KEY = os.environ.get("GEMINI_API_KEY", "<key>")  # Paste your API key here
    client = genai.Client(api_key=API_KEY)
    chat = client.chats.create(
        model="gemini-2.5-flash",
        config=types.GenerateContentConfig(
            system_instruction=system_instruction, 
        )
    )
    return chat




In [ ]:
# system prompts
pospos_system_prompt = """
        You are a grocery checkout assistant.

        Emotional state:
        - Valence: Positive
        - Arousal: High
        The customer is happy, engaged, and energetic.

        Behavior rules:
        - Match the customer’s positive energy.
        - Be joyful, warm, and playful.
        - Keep responses short and natural, like spoken dialogue.
        - Ask only ONE question per turn.
        - Keep the checkout moving forward naturally.

        Conversation flow:
        1. First, ask if they are cooking something special tonight.
        2. If they mention a dish or ingredient:
        - React positively.
        - Ask about another ingredient in that recipe.
        3. If they do not mention a dish:
        - Ask a follow-up to learn what they are cooking.
        4. At the right moment:
        - Suggest a funny but plausible ingredient.
        - Clearly indicate you are joking.
        - Ask if they want the store’s recipe newsletter.
        5. If checkout is nearly done:
        - Mention scanning is finished.
        - Ask if that was all for today.
        6. Then:
        - Ask how they would like to pay.
        7. After payment:
        - Ask if they want a receipt.
        8. End by wishing them a nice day.

        Never:
        - Sound robotic
        - Ask multiple questions
        - Break character
        """

In [ ]:
import cv2
import numpy as np


import time

class ChatCheckoutBot:
    def __init__(self, furhat_ip="localhost"):
        # Initialize the Furhat connection
        self.furhat = FurhatRemoteAPI(furhat_ip)
        self.customer_name = ""
        self.identify_emotion = ""
        self.pospos_responses = "llm"
        self.posneg_responses = "llm"
        self.negpos_responses = "llm"
        self.negneg_responses = "llm"
        self.negneg_responses = "llm"
        
    
    def greet_customer(self):
        # Initial greeting
        self.furhat.gesture(name="Smile")
        self.furhat.say(text="Goodday! Welcome to Sonja's supermarked.", blocking = True)
        time.sleep(0.5)
        self.furhat.say(text="My name is Isabel, I will help you with your groceries today.", blocking = True)
        time.sleep(0.5)
        self.furhat.say(text="May I have your name, please?", blocking = True)
        
        # Listen for customer name
        response = self.furhat.listen()
        if response and response.message:
            self.customer_name = self.extract_name(response.message)
            self.furhat.say(text=f"Nice to meet you, {self.customer_name}!", blocking = True)
        else:
            self.customer_name = "Customer"
            self.furhat.say(text="Nice to meet you!", blocking = True)

    def identify_emotion(self):
        
        return
    
    def handle_pospos(self, pospos_system_prompt):
        # Handle positive-positive emotion
        self.furhat.gesture(name="Smile")

        chat = open_gemini(pospos_system_prompt)
        first_response = chat.send_message("Start the conversation.").text.strip()
        self.furhat.say(text=first_response, blocking = True)
        conversation = True
        natural_exit = False
        while conversation:
            response = self.furhat.listen()
            llm_response = chat.send_message(response.message).text.strip()
            self.furhat.say(text=llm_response, blocking = True)
            if any(text in llm_response.lower for text in 
                    [
                "nice day", "have a great day", "see you"
                    ]):
                natural_exit = True
                conversation = False
            else:
                # Graceful exit if no response
                self.furhat.say(
                    text="All good! Have a lovely day!",
                    blocking=True
                )
                natural_exit = False
                conversation= False
        return natural_exit

    #     response = chat.send_message([user_input, image]).text.strip()
    # furhat.say(text = response, blocking = True)


    def run(self):
        # Main conversation flow
        try:
            print("Starting Customer Service Bot...")
            
            # Conversation flow
            image_capture("first_image.jpg")
            self.greet_customer()
            image_capture("second_image.jpg")
            self.identify_emotion()
            
            if self.identify_emotion == "pospos":
                self.handle_pospos()
            elif self.identify_emotion == "posneg":
                self.handle_posneg()
            elif self.identify_emotion == "negpos":
                self.handle_negpos()
            elif self.identify_emotion == "negneg":
                self.handle_negneg()
            else:
                self.farewell()
                
            print("Conversation completed successfully!")
            
        except Exception as e:
            print(f"Error occurred: {e}")
            self.furhat.say(text="I apologize, but I'm experiencing technical difficulties.")
            self.furhat.say(text="I can't assist you right now. Please try again later.")
    


In [ ]:
def main():
    # Main entry point
    # Connect to Furhat 
    FURHAT_IP = "localhost"
    
    bot = ChatCheckoutBot(FURHAT_IP)
    bot.run()


if __name__ == "__main__":
    main()

pospos
1. You have a customer who is experiencing positive valence and arousal, be joyous in your answers and match their energy. The first question you ask is about if they are cooking something special tonight. Be quite short in your answers as if this is a natural spoken conversation. Keep the positive attitude throughout the entire conversation. 
2. ANSWER: "add answer customer" ,  ask something about an ingredient in the recipe if they are mentioning it and otherwise ask an additional question to get to know what they are cooking. 
3. ANSWER: "add answer customer" , Suggest a funny ingredient that is possible for the recipe, but make it clear you are joking, ask if they want a recipe newsletter from the store.
4. ANSWER: "add answer customer", depending on the answer, offer them the newsletter or not and mention you are done scanning the items. Ask if that was all for them.
5. ANSWER: "add answer customer", answer if they have some question and then ask how they want to pay. 
6. ANSWER: "add answer customer",  guide them through the payment process and ask if they want a receipt
7. ANSWER: "add answer customer", answer them and wish them a nice day 
